# Features del log y de resulados académicos globales
---

En este notebook vamos a calcular un nuevo dataset en el cual se van a calcular nuevas métricas más generales, usando información del log y información global de calificaciones y entregas. 

En concreto , se recopilarán las siguientes métricas:

-   Media de notas EC
-   Proporción de entregas hechas EC
-   Número de logs por mes
-   Número de logs totales
-   Máximo número de días sin conectarse
-   Máximo número de días seguidos con actividad registrada

## Configuración 

In [29]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col, lit, max
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql.functions import count
from pyspark.sql.functions import from_unixtime, date_format, to_date

import os

DATOS_LOG = "/home/carlos/Documentos/TFG/spark-workspace/data/raw/"
DATOS_AC = "/home/carlos/Documentos/TFG/spark-workspace/data/datasets"
DATOS_DESTINO = "/home/carlos/Documentos/TFG/spark-workspace/data/datasets"

courseid_ip = 8683


# Crear sesión Spark
spark = SparkSession.builder \
    .appName("Creacion de metricas de los cuestionarios") \
    .master("local[*]") \
    .config("spark.sql.shuffle.partitions", "8") \
    .getOrCreate()

## Carga de datos

In [25]:
# Cargar datos log
#==========================================================
df_log_ip = spark.read.parquet(f"{DATOS_LOG}/log_ip_cmi.parquet")
df_log_ip = df_log_ip.drop("courseid")

#Cargar datos alumnos
df_alumnos_ip = spark.read.parquet(f"{DATOS_LOG}/alumnos_ip_cmi.parquet")

#Quedarnos solo con las entradas de la ventana de observación
df_log_date = df_log_ip.withColumn("date", to_date(from_unixtime(col("timecreated"))))

df_log_obs = df_log_date.filter(
      (col("date") >= "2023-09-01") & (col("date") <= "2023-11-21")) \
.orderBy("date")

# Nos quedamos solo con los accesos al curso (No hace falta unir con alummnos matriculados ya que los logs estan calculados solo para ellos)
df_accesos_alumnos = df_log_obs.filter(
(col("component") == "core") & # Indicar el módulo de moodle que ha registrado la entrada, en este caso al tratarse de un acceso al aula virtual debe ser core
(col("action") == "viewed") & # Indicar la acción que se ha realizado, en este caso un acceso al aula virtual
(col("contextinstanceid") == courseid_ip) #Pasar el id del curso como contextinstanceid , dado que así indicamos que el acceso ha sido al curso y no a un recurso del mismo
)

print(f"Esquema de datos recogidos del log:\n")
df_accesos_alumnos.printSchema()

# Cargar datos académicos
#==========================================================

df_entregas = spark.read.parquet(f"{DATOS_AC}/dataset_1.1.parquet")
df_notas = spark.read.parquet(f"{DATOS_AC}/dataset_1.2.parquet")


#Nos quedamos solo con el número de entregas por usuario y el userid
df_entregas = df_entregas.select("userid", "num_entregas") 

#Nos quedamos solo con la media de notas de actividades del usuario y el userid
df_notas = df_notas.select("userid", "nota_media")

print(f"Esquema de datos de entregas de actividades:\n")
df_entregas.printSchema()
print(f"Esquema de datos recogidos notas de actividades:\n")
df_notas.printSchema()


Número de tuplas en df_accesos_alumnos_gb: 200
Esquema de datos recogidos del log:

root
 |-- userid: string (nullable = true)
 |-- timecreated: long (nullable = true)
 |-- eventname: string (nullable = true)
 |-- component: string (nullable = true)
 |-- action: string (nullable = true)
 |-- contextinstanceid: long (nullable = true)
 |-- date: date (nullable = true)

Esquema de datos de entregas de actividades:

root
 |-- userid: string (nullable = true)
 |-- num_entregas: long (nullable = true)

Esquema de datos recogidos notas de actividades:

root
 |-- userid: string (nullable = true)
 |-- nota_media: double (nullable = true)



## Media de notas EC y proporción de entregas

In [14]:
df_activities_features = df_notas.join(df_entregas, on="userid", how="inner") 

# Verificación de número de alumnos tras el inner join
# print(f"Cantidad de alumnos: {df_activities_features.count()}")


#Dividir las entregas hechas por el total (7 en esta asignatura) para tener la proporción
df_activities_features = df_activities_features.withColumn(
    "prop_entregas_hechas",
    col("num_entregas") / lit(7)
).drop("num_entregas")

df_activities_features.show(300, truncate=False)


+----------------------------------------------------------------+------------------+--------------------+
|userid                                                          |nota_media        |prop_entregas_hechas|
+----------------------------------------------------------------+------------------+--------------------+
|e1f1d0f48ca77093f9d66cefd325504245277db3e6c14504a25fda693e82a393|8.416666666666666 |0.8571428571428571  |
|b5de2bb5b8538b199d6b3f0ecb32daa8a9d730ccc484dba45f756a59254c6dbe|8.892857142857142 |1.0                 |
|90a634296aff946e9d045997d512d2b77dbc01880715c1e179eafed0ec78378c|8.380952857142857 |1.0                 |
|b6b2a12e84ea8203775195ed2bb4e99c5788053782b0bdae24fa6adcae487d22|9.619047142857143 |1.0                 |
|fd96e32a94a932f45eb32933d9ffeb71f4addf9153a76b4e5dec57982ec71bfb|7.0               |0.8571428571428571  |
|ad2273914219245f3a1d76fa50e1e719d5342979b9bbca09f5e36e9f4be14d7d|9.928571428571429 |1.0                 |
|dd7af0da56a7f883acf1ca25d39672bd045b

## Métricas del log

### Dias totales globales y por mes

In [27]:



df_accesos_alumnos.select("userid", "date")
num_accesos = df_accesos_alumnos.count()
print(f"Número de accesos al curso hechos en la ventana de observación por alumnos matriculados: {num_accesos}")

# Agrupar por usuario y obtener número de accesos por alumno matriculado en el periodo considerado
df_accesos_usuario = df_accesos_alumnos.groupBy("userid").agg(count("*").alias("num_accesos"))
print(f" Número de usuarios en el df: {df_accesos_usuario.count()}")

# Filter and show distinct dates for September, and then count to obtain the metric
df_accesos_usuario_sept_filtered = df_accesos_alumnos.filter(
      (date_format(col("date"), "yyyy-MM") == "2023-09")
)
# df_accesos_usuario_sept_filtered.select("date").distinct().orderBy("date").show(truncate=False)
df_accesos_usuario_sept = df_accesos_usuario_sept_filtered.groupBy("userid").agg(count("*").alias("num_accesos_sept"))
# df_accesos_usuario_sept.show(5, truncate=False)

# Filter and show distinct dates for October, and then count to obtain the metric
df_accesos_usuario_oct_filtered = df_accesos_alumnos.filter(
      (date_format(col("date"), "yyyy-MM") == "2023-10")
)
# df_accesos_usuario_oct_filtered.select("date").distinct().orderBy("date").show(truncate=False)
df_accesos_usuario_oct = df_accesos_usuario_oct_filtered.groupBy("userid").agg(count("*").alias("num_accesos_oct"))
# df_accesos_usuario_oct.show(5,truncate=False)

# Filter and show distinct dates for November, and then count to obtain the metric
df_accesos_usuario_nov_filtered = df_accesos_alumnos.filter(
      (col("date") >= "2023-11-01") & (col("date") <= "2023-11-21")
)
# df_accesos_usuario_nov_filtered.select("date").distinct().orderBy("date").show(truncate=False)
df_accesos_usuario_nov = df_accesos_usuario_nov_filtered.groupBy("userid").agg(count("*").alias("num_accesos_nov"))
# df_accesos_usuario_nov.show(5,truncate=False)

df_accesos_feature = df_accesos_usuario.join(df_accesos_usuario_sept, on="userid", how="left") \
      .join(df_accesos_usuario_oct, on="userid", how="left") \
      .join(df_accesos_usuario_nov, on="userid", how="left")

df_accesos_feature.show(300, truncate=False)

print(f"Número de usuarios: {df_accesos_feature.count()}")


Número de accesos al curso hechos en la ventana de observación por alumnos matriculados: 32259
 Número de usuarios en el df: 200
+----------------------------------------------------------------+-----------+----------------+---------------+---------------+
|userid                                                          |num_accesos|num_accesos_sept|num_accesos_oct|num_accesos_nov|
+----------------------------------------------------------------+-----------+----------------+---------------+---------------+
|3b8d431cbee3182d06225f9d5ab51f5806e8042f145e0ad0e37d98f56ae78f3d|84         |26              |31             |27             |
|368093a57fe640879a9fc57ecb7e2c846b7dadf19620bfc9c4c001daeaf9af0f|69         |24              |24             |21             |
|b10bfb431460e96f99322635d8c44d0fb4f74302c61334119ab1c842e049fac1|97         |32              |44             |21             |
|e86f7c2d50f4fa9ee5b3f8bf9a25af4c9ef573fa28a0c8c50df815971efa0529|111        |46              |42      

### Máximo número de días sin acceder 

In [30]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col, datediff, lag


# Paso 1: Crear ventana por alumno, ordenada por fecha de acceso
ventana = Window.partitionBy("userid").orderBy("date")

# Paso 2: Calcular diferencia de días entre fechas consecutivas
df_con_diff = df_accesos_alumnos.withColumn("fecha_anterior", lag("date").over(ventana))
df_con_diff = df_con_diff.withColumn("dias_sin_acceso", datediff("date", "fecha_anterior"))

# Paso 3: Obtener el máximo salto de días por alumno
dias_maximos = df_con_diff.groupBy("userid").agg(
    max("dias_sin_acceso").alias("max_dias_sin_acceso")
)

dias_maximos.show(5, truncate = False)

+----------------------------------------------------------------+-------------------+
|userid                                                          |max_dias_sin_acceso|
+----------------------------------------------------------------+-------------------+
|006b0e7bd07cec05e0952cb61c30893f6d30d7962f9efc99d0f041f6fadcc320|7                  |
|00ded60939d4949cc46e46e865b25d3f11756733cf946087710c61eda02729e1|4                  |
|05912200993a87a89df1a6ca9ac3d6493e2c4cc178760d8ee1da41033ac01b3e|5                  |
|073b1d0ee1d3857d50ea87087b25bbc6f5dbdbd2e94bcf52b89c48afa37e8c16|3                  |
|080b2c8b65e9d941f12e62b7d2b9fa22b669f06aeed07df5683fdf93a799204d|7                  |
+----------------------------------------------------------------+-------------------+
only showing top 5 rows


### Máximo número de días consecutivos con actividad

In [32]:


from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, datediff, lit, col, count, max

# 1) Mantener un único registro por día y alumno
df_dias_unicos = df_accesos_alumnos.select("userid", "date").distinct()

# 2) Para cada alumno, generar un identificador constante dentro de cada racha
ventana_orden = Window.partitionBy("userid").orderBy("date")

df_rachas = (
    df_dias_unicos
        .withColumn("idx", row_number().over(ventana_orden))
        .withColumn("dias_desde_epoch", datediff(col("date"), lit("1970-01-01")))
        .withColumn("grupo_racha", col("dias_desde_epoch") - col("idx"))
)

# 3) Contar la longitud de cada racha y quedarnos con la máxima
df_longitud_rachas = (
    df_rachas
        .groupBy("userid", "grupo_racha")
        .agg(count("*").alias("longitud_racha"))
)

dias_consecutivos_max = (
    df_longitud_rachas
        .groupBy("userid")
        .agg(max("longitud_racha").alias("max_dias_consecutivos_accediendo"))
)

dias_consecutivos_max.show(5, truncate=False)


+----------------------------------------------------------------+--------------------------------+
|userid                                                          |max_dias_consecutivos_accediendo|
+----------------------------------------------------------------+--------------------------------+
|006b0e7bd07cec05e0952cb61c30893f6d30d7962f9efc99d0f041f6fadcc320|6                               |
|00ded60939d4949cc46e46e865b25d3f11756733cf946087710c61eda02729e1|13                              |
|05912200993a87a89df1a6ca9ac3d6493e2c4cc178760d8ee1da41033ac01b3e|5                               |
|073b1d0ee1d3857d50ea87087b25bbc6f5dbdbd2e94bcf52b89c48afa37e8c16|13                              |
|080b2c8b65e9d941f12e62b7d2b9fa22b669f06aeed07df5683fdf93a799204d|3                               |
+----------------------------------------------------------------+--------------------------------+
only showing top 5 rows


## Unir todas las métricas, unir con etiquetas y exportar

In [47]:
from os import truncate
import pandas as pd

df_etiquetas = spark.read.parquet("/home/carlos/Documentos/TFG/spark-workspace/data/datasets/etiquetas/etiquetas_abandono_entregas.parquet")


df_features_globales = df_activities_features.join(df_accesos_feature, on="userid", how="left") \
      .join(dias_maximos, on="userid", how="left") \
      .join(dias_consecutivos_max, on="userid", how="left")
      
df_features_globales = df_features_globales.fillna(0)

print(f"Esquema de datos de las features globales:\n")

df_features_globales = df_features_globales.withColumnRenamed("nota_media", "nota_media_actividades") \
      .withColumnRenamed("prop_entregas_hechas", "proporcion_actividades_hechas") 

df_features_globales = df_features_globales.join(df_etiquetas, on="userid", how="left") \
      .fillna(0) \
      .select("userid", "nota_media_actividades", "proporcion_actividades_hechas", "num_accesos", "num_accesos_sept", "num_accesos_oct", "num_accesos_nov", "max_dias_sin_acceso", "max_dias_consecutivos_accediendo", "abandona")
      
df_features_globales.printSchema()
df = df_features_globales.toPandas()
display(df.head())
# Guardar el DataFrame como un archivo Parquet
df.to_parquet(f"{DATOS_DESTINO}/dataset_2.0.parquet", index=False)

print("Cantidad de valores nulos por columna:")
print(df.isnull().sum())

print(f"Cantidad de tuplas en el DataFrame: {len(df)}")


Esquema de datos de las features globales:

root
 |-- userid: string (nullable = true)
 |-- nota_media_actividades: double (nullable = false)
 |-- proporcion_actividades_hechas: double (nullable = false)
 |-- num_accesos: long (nullable = true)
 |-- num_accesos_sept: long (nullable = true)
 |-- num_accesos_oct: long (nullable = true)
 |-- num_accesos_nov: long (nullable = true)
 |-- max_dias_sin_acceso: integer (nullable = true)
 |-- max_dias_consecutivos_accediendo: long (nullable = true)
 |-- abandona: integer (nullable = true)



,userid,nota_media_actividades,proporcion_actividades_hechas,num_accesos,num_accesos_sept,num_accesos_oct,num_accesos_nov,max_dias_sin_acceso,max_dias_consecutivos_accediendo,abandona
0,e1f1d0f48ca77093f9d66cefd325504245277db3e6c145...,8.416667,0.857143,183,63,81,39,5,12,0
1,b5de2bb5b8538b199d6b3f0ecb32daa8a9d730ccc484db...,8.892857,1.000000,149,53,58,38,6,7,0
2,90a634296aff946e9d045997d512d2b77dbc01880715c1...,8.380953,1.000000,102,31,40,31,6,6,1
3,b6b2a12e84ea8203775195ed2bb4e99c5788053782b0bd...,9.619047,1.000000,175,74,65,36,6,7,0
4,fd96e32a94a932f45eb32933d9ffeb71f4addf9153a76b...,7.000000,0.857143,111,23,57,31,7,6,0


Cantidad de valores nulos por columna:
userid                              0
nota_media_actividades              0
proporcion_actividades_hechas       0
num_accesos                         0
num_accesos_sept                    0
num_accesos_oct                     0
num_accesos_nov                     0
max_dias_sin_acceso                 0
max_dias_consecutivos_accediendo    0
abandona                            0
dtype: int64
Cantidad de tuplas en el DataFrame: 201
